In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, r2_score
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("Data/TrainAndValid.csv", low_memory=False, parse_dates=["saledate"])

In [3]:
df.head().T

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000.0,57000.0,10000.0,38500.0,11000.0
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3.0,3.0,3.0,3.0,3.0
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68.0,4640.0,2838.0,3486.0,722.0
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   412698 non-null  int64         
 1   SalePrice                 412698 non-null  float64       
 2   MachineID                 412698 non-null  int64         
 3   ModelID                   412698 non-null  int64         
 4   datasource                412698 non-null  int64         
 5   auctioneerID              392562 non-null  float64       
 6   YearMade                  412698 non-null  int64         
 7   MachineHoursCurrentMeter  147504 non-null  float64       
 8   UsageBand                 73670 non-null   object        
 9   saledate                  412698 non-null  datetime64[ns]
 10  fiModelDesc               412698 non-null  object        
 11  fiBaseModel               412698 non-null  object        
 12  fi

In [5]:
df.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

In [6]:
# Feature engineering
df["year"] = df["saledate"].dt.year
df["month"] = df["saledate"].dt.month
df["day"] = df["saledate"].dt.day
df["weekday"] = df["saledate"].dt.weekday

df.drop("saledate", axis=1, inplace=True)

In [9]:
# Filling Missing data
def smart_impute(df):
    df = df.copy()
    missing_cols = {}
    filled_cols = {}

    for col in df.columns:
        missing_cols[f"missing_{col}"] = df[col].isna().astype(int)

        if df[col].dtype == "object" or str(df[col].dtype).startswith("string"):
            filled_cols[col] = df[col].fillna("missing")
        else:
            filled_cols[col] = df[col].fillna(0)

    df_out = pd.concat([pd.DataFrame(filled_cols), pd.DataFrame(missing_cols)], axis=1)
    return df_out

In [10]:
df = smart_impute(df)

In [11]:
df.head().T

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000.0,57000.0,10000.0,38500.0,11000.0
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
...,...,...,...,...,...
missing_Steering_Controls,0,0,1,1,1
missing_year,0,0,0,0,0
missing_month,0,0,0,0,0
missing_day,0,0,0,0,0


In [12]:
# Handling Strings
df = pd.get_dummies(df)

In [16]:
# Reducing Data
def drop_zero_columns(df):
    return df.loc[:, (df != 0).any(axis=0)]

df = drop_zero_columns(df)

In [22]:
# Splitting Data
X = df.drop("SalePrice", axis=1)
y = df["SalePrice"]

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5
)

In [28]:
model = CatBoostRegressor(verbose=False)
model.fit(X_train, y_train)

In [29]:
val_pred = model.predict(X_val)
test_pred = model.predict(X_test)

In [33]:
metrics = {
    "MAE": mean_absolute_error,
    "RMSE": lambda y,a: np.sqrt(mean_squared_error(y,a)),
    "R2": r2_score
}

df_eval = pd.DataFrame(
    {m: [
        f(y_train, model.predict(X_train)),
        f(y_val,   model.predict(X_val))
    ] for m,f in metrics.items()},
    index=["Training", "Validation"]
).T

print(df_eval.to_string())

         Training   Validation
MAE   5140.757808  5245.612062
RMSE  7603.157672  7807.121474
R2       0.892121     0.885142
